### Imports

In [3]:
import networkx as nx
import random
import time
import itertools
import math
import multiprocessing as mp
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import os

### Graph loading and metrics calculation

In [4]:
# Function to read the data from the cytoscape js file
def cyjs2graph(cyjs_file_name):
    cyjson = json.load(open(cyjs_file_name))
    name_from_id = {}
    for node in cyjson["elements"]["nodes"]:
        name_from_id[node['data']['id']] = node['data']['name']
    edge_list = []
    for edge in cyjson["elements"]["edges"]:
        src_id = edge['data']['source']
        src_name = name_from_id[src_id]
        tgt_id   = edge['data']['target']
        tgt_name = name_from_id[tgt_id]
        edge_list.append([src_name, tgt_name])
    graph = nx.from_edgelist(edge_list, create_using=nx.Graph, )
    return graph

Formulas for metrics calculation (based on [Cytohubba](https://bmcsystbiol.biomedcentral.com/articles/10.1186/1752-0509-8-S4-S11#Sec7)).

**** 

#### Local-based Methods:

1. **Degree method (Deg)**
   
$Deg(v) = |N(v)|$

2. **Maximum Neighborhood Component (MNC)**

$MNC(v) = |V(MC(v))|$, where $MC(v)$ is a maximum connected component of the $G[N(v)]$ and $G[N(v)]$ is the induced subgraph of $G$ by $N(v)$.

3. **Density of Maximum Neighborhood Component (DMNC)**

Based on MNC, Lin et. al. proposed $DMNC(v) = |E(MC(v))|/ |V(MC(v))| \epsilon$, where $\epsilon = 1.7$.

4. **Maximal Clique Centrality (MCC)**

Given a node v, the MCC of v is defined as $MCC(v) = \sum_{C\in S(v)}{(|C|-1)!}$, where $S(v)$ is the collection of maximal cliques which contain $v$, and $(|C|-1)!$ is the product of all positive integers less than $|C|$. If there is no edge between the neighbors of the node $v$, then $MCC(v)$ is equal to its degree.

****

#### Global-based methods:

5. **Closeness (Clo)**

$Clo(v) = \sum_{w\in V}{\frac{1}{dist(v,w)}}$

6. **EcCEntricity (EC)**

$EC(v) = \frac{|V(C(v))|}{|V|} \times \frac{1}{\max \{ \text{dist}(v, w) : w \in C(v) \}}$

7. **Radiality (Rad)**

$Rad(v) = \frac{|V(C(v))|}{|V|} \times \frac{\sum_{w\in C(v)}{(\Delta_{C(v)} + 1 - dist(v,w))}}{max\{dist(v,w):w\in C(v)\}}$, where $\Delta_{C(v)}$ is the maximum distance between any two vertices of the component $C(v)$.

8. **BottleNeck (BN)**

Let $T_{s}$ be a shortest path tree rooted at node $s$, $BN(v) = \sum_{s\in V}{p_s (v)}$, where $p_{s}(v) = 1$ if more than $|V(T_{s})|/4$ paths from node $s$ to other nodes in $T_s$ meet at the vertex $v$; otherwise $p_s(v) = 0$.

9. **Stress (Str)**

$Str(v) = \sum_{s \neq t \neq v \in C(v)}{\sigma_{st}(v)}$, where $\sigma_{st}(v)$ is the number of shortest paths from node $s$ to node $t$ which use the node $v$.

10. **Betweenness (BC)**

$BC(v) = \sum_{s \neq t \neq v \in C(v)}{\sigma_{st}(v)}$, where $\sigma_{st}$ is the number of shortest paths from node s to node t.

11. **Edge Percolated Component (EPC)**

Given a threshold ($0\leq$ the threshold $\leq1$), we create $1000$ reduced networks by assigning a random number between $0$ and $1$ to every edge and remove edges if their associated random numbers are less than the threshold.

Let the $G_k$ be the reduced network generated at the $k$th time reduced process. If nodes $u$ and $v$ are connected in $G_k$, set $\delta_{vt}^k$
to be 1; otherwise $\delta_{vt}^k=0$. For a node $v$ in $G$, $EPC(v)$ is defined as $EPC(v)=\frac{1}{|V|} \sum^{1000}_{k=1}\sum_{t\in V} \delta_{vt}^k$.  

******

In [19]:
###### LOCAL BASED METHODS 

def MNC(G, v) -> float:
    """
    MNC(v) = |V(MC(v))|, where MC(v) is a maximum connected component
    of the G[N(v)] and G[N(v)] is the induced subgraph of G by N(v).
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
    """
    
    induced_subgraph = G.subgraph(list(G.neighbors(v)))
    components = nx.connected_components(induced_subgraph)
    max_component = max(components, key=len)
    
    return len(max_component)


def DMNC(G, v, epsilon=1.7) -> float:
    """
    Based on MNC, Lin et. al. proposed DMNC(v) = |E(MC(v))|/ |V(MC(v))|ε, where ε = 1.7 [10].
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
        epsilon (float): parameter from Lin et. al., 1 < epsilon < 2
    """
    
    induced_subgraph = G.subgraph(list(G.neighbors(v)))
    components = nx.connected_components(induced_subgraph)
    max_component = max(components, key=len)
    # finding induced subgraph of the maximum component
    mc_subgraph = G.subgraph(max_component)
    num_nodes = mc_subgraph.number_of_nodes()
    num_edges = mc_subgraph.number_of_edges()
    if num_nodes == 0:
        return 0
        
    return num_edges / (num_nodes ** epsilon)


def MCC(G, v) -> float:
    """
    MCC(v) = sum_{C from S(v)}(|C| - 1)!, where S(v) is the 
    collection of maximal cliques which contain v.
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
    """
    
    cliques = list(nx.find_cliques(G))
    cliques_with_v = [clique for clique in cliques if v in clique]
    mcc = 0
    for clique in cliques_with_v:
        size_of_clique = len(clique)
        if size_of_clique > 1:
            mcc += math.factorial(size_of_clique - 1)
    # if there are no cliques, MCC is equal to the degree of the node
    if not cliques_with_v:
        return G.degree(v)
        
    return mcc


###### GLOBAL BASED METHODS 

def closeness_centrality(G, v) -> float:
    """
    Clo(v) = sum_{w in V}{1/dist(v,w)}
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
    """
    
    lengths = nx.single_source_shortest_path_length(G, v)
    closeness = sum(1 / dist for node, dist in lengths.items() if dist > 0)
    
    return closeness


def EC(G, v) -> float:
    """
    EcCentricity (EC), see formula in the methods' description.
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
    """
        
    component = set(nx.node_connected_component(G, v))
    term1 = len(component) / len(G)
    distances = {node: dist for node, dist in nx.single_source_shortest_path_length(G, v).items() if node in component}
    max_distance = max(distances.values(), default=0)
    term2 = 1 / max_distance if max_distance > 0 else 0
    
    return term1 * term2


def radiality(G, v) -> float:
    """
    Radiality (Rad), see formula in the methods' description.
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
    """
    
    component = set(nx.node_connected_component(G, v))
    num_nodes_component = len(component)
    num_nodes_graph = len(G)
    
    distances = {node: dist for node, dist in nx.single_source_shortest_path_length(G, v).items() if node in component}
    max_dist = max(distances.values(), default=0)
    max_diameter = max((nx.single_source_shortest_path_length(G, node).get(v, 0) for node in component), default=0)
    sum_term = sum(max_diameter + 1 - dist for dist in distances.values())
    
    if max_dist == 0:
        return 0
    term1 = num_nodes_component / num_nodes_graph
    term2 = sum_term / max_dist
    
    return term1 * term2


def bottleneck(G, v) -> float:
    """
    BottleNeck (BN), see formula in the methods' description.
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
    """
    
    bn_value = 0
    for s in G.nodes():
        T_s = nx.single_source_shortest_path(G, s)
        path_count = sum(1 for paths in T_s.values() if v in paths and len(paths) > 1)
        if path_count > len(G.nodes) / 4:
            bn_value += 1
    
    return bn_value


def stress_centrality(G, v) -> float:
    """
    Stress (Str), see formula in the methods' description.
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
    """

    stress_value = 0
    for s, t in itertools.combinations(G.nodes(), 2):
        if s != v and t != v:
            paths = list(nx.all_shortest_paths(G, s, t))
            stress_value += sum(1 for path in paths if v in path[1:-1])
    
    return stress_value


def edge_percolated_component(G, v, threshold=0.5, iterations=1000) -> float:
    """
    Edge Percolated Component (EPC), see formula in the methods' description.
    Args:
        G (networkx graph): networkx graph object
        v (int): node id of the target node
        threshold (float): 
        iterations (int): 
    """
    
    num_nodes = len(G.nodes)
    epc_value = 0
    
    for _ in range(iterations):
        G_k = G.copy()
        for edge in list(G_k.edges):
            if random.random() < threshold:
                G_k.remove_edge(*edge)
        
        connected_nodes = nx.node_connected_component(G_k, v)
        epc_value += len(connected_nodes)
    
    return epc_value / (iterations * num_nodes)


def calc_metrics(G) -> dict:
    """
    This function receives a graph for the PPI network and returns 
    topology metrics. Metrics are based on the Cytohubba, a Cytoscape plugin.

    Args:
        G (networkx graph): networkx graph object
    """
    
    node_metrics_dict = dict()
    deg = nx.degree_centrality(G)
    bw = nx.betweenness_centrality(G)
    for v in G.nodes():
        metrics_dict = {
            # local based methods: only consider the direct neighbourhood of a vertex
            'degree': deg[v],
            'MNC': MNC(G, v),
            'DMNC': DMNC(G, v),
            'MCC': MCC(G, v),
            # global based methods: methods based on shortest paths
            'closeness': nx.closeness_centrality(G, v),
            'eccentricity': EC(G, v),
            'radiality': radiality(G, v),
            'bottleneck': bottleneck(G, v),
            'stress': stress_centrality(G, v),
            'betweenness': bw[v],
            'EPC': edge_percolated_component(G, v),
        }
        node_metrics_dict[v] = metrics_dict
        
    return node_metrics_dict

#### Example run with small network:

In [27]:
G = nx.karate_club_graph()
G.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33))

In [28]:
mdict = calc_metrics(G)
metrics_df = pd.DataFrame(mdict).T
metrics_df.head()

,degree,MNC,DMNC,MCC,closeness,eccentricity,radiality,bottleneck,stress,betweenness,EPC
0,0.484848,10.0,0.279337,68.0,0.568966,0.333333,26.000000,33.0,843.0,0.437635,0.835735
1,0.272727,8.0,0.349887,55.0,0.485294,0.333333,22.666667,2.0,133.0,0.053937,0.823706
2,0.303030,7.0,0.402463,55.0,0.559322,0.333333,25.666667,24.0,241.0,0.143657,0.828441
3,0.181818,6.0,0.475492,50.0,0.464789,0.333333,21.666667,1.0,29.0,0.011909,0.817941
4,0.090909,3.0,0.308975,4.0,0.379310,0.250000,20.750000,1.0,1.0,0.000631,0.669265


#### Our networks:

In [20]:
# Change to a path to the target network
path = "../data/prots_data/lcc/ab16_imp.cyjs"
# Path, where to save the metrics csv
mdict_name = "prot_ab16_imp_metrics.csv"

G = cyjs2graph(path)
mdict = calc_metrics(G)
metrics_df = pd.DataFrame(mdict).T
metrics_df.to_csv(mdict_name)